Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

In [ ]:
import os
import os.path
import pandas as pd

datadir = "publicdata"

In [ ]:
def meta2country_aggs(path):
    df = pd.read_csv(path)
    df.columns=['code', 'region', 'income', 'notes', 'country', 'extra']
    df.set_index('code',inplace=True)
    notaggs = df[pd.notna(df.region)].drop(['notes','extra'], axis=1)
    aggs = df[pd.isna(df.region)].drop(['region','notes','income','extra'], axis=1)
    aggs.columns=['aggregate']
    notaggs = notaggs[['country', 'region', 'income']].copy()
    return notaggs, aggs

def ind2df(path, aggregates, indicator):
    df = pd.read_csv(path, skiprows=4)
    df.rename({'Country Name': 'country', 'Country Code': 'code', 
               'Indicator Name': 'indicator', 'Indicator Code': 'icode'}, 
               axis=1, inplace=True)
    df.set_index('code', inplace=True)
    df.drop(list(aggregates.index), axis=0, inplace=True)
    #print(len(df))
    for column_label in list(df.columns)[-1::-1]:
        found_non_null = False
        if df[column_label].isnull().all():
            #print("Column {} all NaN".format(column_label))
            df.drop(column_label, axis=1, inplace=True)
        else:
            break
    df.drop(['country', 'indicator', 'icode'], axis=1, inplace=True)
    df.reset_index(inplace=True)
    melted = df.melt(id_vars='code', var_name='year', value_name=indicator)
    melted = melted.astype({'year': int})
    melted.set_index(['year', 'code'], inplace=True)
    melted.sort_index(inplace=True)
    return melted

def subsetind(oneindicator, year=2016, code=['USA','CAN']):
    """ Subset a single indicator in both year(s) and country code(s)
    """
    levellist = []
    if isinstance(year, int):
        yearsubset = oneindicator.xs(key=year, level='year').copy()
    else:
        yearsubset = oneindicator.loc[year].copy()
        levellist.append('year')
    
    yearsubset.reset_index(inplace=True)
    
    if isinstance(code, str):
        subset = yearsubset[yearsubset.code == code].copy()
    else:
        subset = yearsubset[yearsubset.code.apply(lambda x: x in code)].copy()
    levellist.append('code')
    subset.set_index(levellist, inplace=True)
    subset.sort_index(inplace=True)
    
    return subset

def build_indicators(indlist, conversions, metalist, yearlist, codelist, 
                     aggregates, countrydf, datadir):
    conversions = conversions.copy()
    indicator = indlist[0]
    convscale, convround = conversions.pop(0)
    path = os.path.join(datadir, indicator+'.csv')
    oneindicator = ind2df(path, aggregates, indicator)
    df = subsetind(oneindicator, year=yearlist, code=codelist)
    df[indicator] = round(df[indicator] / convscale, convround)
    for indicator in indlist[1:]:
        convscale, convround = conversions.pop(0)
        path = os.path.join(datadir, indicator+'.csv')
        oneindicator = ind2df(path, aggregates, indicator)
        nextdf = subsetind(oneindicator, year=yearlist, code=codelist)
        nextdf[indicator] = round(nextdf[indicator] / convscale, convround)
        df = pd.concat([df, nextdf], axis=1)

    countrydf2 = countrydf.loc[codelist].copy()
    countrydf2.reset_index(inplace=True)

    metalist2 = ['code'] + metalist
    metasub = countrydf2.loc[:, metalist2].copy()
    metasub

    df.reset_index(inplace=True)

    result = pd.merge(metasub, df, on='code', how='inner')
    if isinstance(yearlist, int):
        result = result.set_index('code')
    else:
        result = result.set_index(['year', 'code'])
    result = result.sort_index()
    return result


In [ ]:
path = os.path.join(datadir, "worldbank", "countrymeta.csv")
countrymeta, aggregates = meta2country_aggs(path)

In [ ]:
path = os.path.join(datadir, 'worldbank', 'land.csv')
landindicator = ind2df(path, aggregates, 'land')
lastyear = list(landindicator.index.levels[0])[-1]
latestland = landindicator.xs(key=lastyear, level='year')
countrydf = countrymeta.join(latestland, how="left")


In [ ]:
path = os.path.join(datadir, "topnames.csv")
topnames0 = pd.read_csv(path)
topnames = topnames0.set_index(['year', 'sex'])


In [ ]:
indlist = ['pop', 'gdp', 'life']
conversions = [(1000000, 2), (1000000000, 2), (1, 2)]
metalist = ['country', 'land']
yearlist = 2017
codelist = ['CHN', 'IND', 'USA', 'GBR']

inddatadir = os.path.join(datadir, "worldbank")
df = build_indicators(indlist, conversions, metalist, yearlist, codelist, 
                      aggregates, countrydf, inddatadir)
df.reset_index(inplace=True)
newcountry = df['country'].combine(df['code'], lambda x, y: "{}--{}".format(y, x))
df = df.drop(['code','country'], axis=1)
df['country'] = newcountry
df.set_index('country', inplace=True)
df


In [ ]:
df.to_csv(os.path.join(datadir, "mashup1.csv"), index=True)

In [ ]:
path = os.path.join(datadir, "tourism2.csv")
metropolis = pd.read_csv(path)
metropolis = metropolis[['date','visitors', 'roomsavail', 'occupancy', 'gamingrevenue']].copy()
metropolis.columns = ['date', 'visitors', 'rooms', 'occupancy', 'revenue']
metropolis2 = metropolis.sort_values('date')
#lasvegas2.head()
metropolishead = metropolis.head()

In [ ]:
metropolis2.to_csv(os.path.join(datadir, "metropolis.csv"), index=False)

In [ ]:
indlist = ['pop', 'gdp', 'exports']
conversions = [(1000000, 2), (1000000000, 2), (1000000000, 2)]
metalist = []
yearlist = 2017
codelist = ['CHN', 'IND', 'GBR']

inddatadir = os.path.join(datadir, "worldbank")
df = build_indicators(indlist, conversions, metalist, yearlist, codelist, 
                      aggregates, countrydf, inddatadir)
messy1 = df.reset_index()
messy2 = messy1.melt(id_vars='code', 
                     value_vars = ['pop', 'gdp', 'exports'],
                     var_name = 'indicator', value_name = 'value')
messy3 = messy2.sort_values('code')
mult_rows = messy3.reset_index().drop('index', axis=1)

In [ ]:
mult_rows.to_csv(os.path.join(datadir, "mult_rows.csv"), index=False)

In [ ]:
indlist = ['pop', 'gdp']
conversions = [(1000000, 2), (1000000000, 2)]
metalist = []
yearlist = [2015, 2017]
codelist = ['CHN', 'IND', 'GBR']

inddatadir = os.path.join(datadir, "worldbank")
df = build_indicators(indlist, conversions, metalist, yearlist, codelist, 
                      aggregates, countrydf, inddatadir)

df2 = df.stack()
mult_rows2 = df2.reset_index()
mult_rows2.columns = ['year', 'code', 'indicator', 'value']
mult_rows2.to_csv(os.path.join(datadir, "mult_rows2.csv"), index=False)

In [ ]:
indlist = ['pop']
conversions = [(1000000, 2)]
metalist = []
yearlist = [2014, 2015, 2016, 2017]
codelist = ['CHN', 'IND', 'USA', 'GBR']

inddatadir = os.path.join(datadir, "worldbank")
df = build_indicators(indlist, conversions, metalist, yearlist, codelist, 
                      aggregates, countrydf, inddatadir)
#df
df2 = df.reset_index()
df3 = df2.pivot(index='code', columns='year')
df4 = df3.droplevel(0, axis=1)
df4.columns = ['pop' + str(x) for x in list(df4.columns)]
pop_columns = df4.reset_index()
pop_columns.to_csv(os.path.join(datadir, "pop_columns.csv"), index=False)

In [ ]:
indlist = ['pop', 'gdp']
conversions = [(1000000, 2), (1000000000, 2)]
metalist = []
yearlist = [2015, 2016, 2017]
codelist = ['CHN', 'IND', 'USA', 'GBR']

inddatadir = os.path.join(datadir, "worldbank")
df = build_indicators(indlist, conversions, metalist, yearlist, codelist, 
                      aggregates, countrydf, inddatadir)
df2 = df.unstack(level='year')
df2.columns = ["{}{}".format(x,y) for x,y in df2.columns.tolist()]
popgdp_columns = df2
popgdp_columns.to_csv(os.path.join(datadir, "popgdp_columns.csv"), index=True)

In [ ]:
path = os.path.join(datadir, "topnames.csv")
topnames = pd.read_csv(path)
topnames.set_index(['year', 'sex'], inplace=True)
topnames.sort_index(level='year', ascending=False, inplace=True)
topnames.reset_index(inplace=True)

topfemale = topnames.loc[topnames.sex == 'Female', ['year', 'name', 'count']]
topfemale.reset_index(inplace=True)
topfemale.drop(axis=1, columns=['index'],inplace=True)
topmale = topnames.loc[topnames.sex == 'Male', ['year', 'name', 'count']]
topmale.reset_index(inplace=True)
topmale.drop(axis=1, columns=['index'],inplace=True)

topmale.to_csv(os.path.join(datadir, "topmale.csv"), index=False)
topfemale.to_csv(os.path.join(datadir, "topfemale.csv"), index=False)

In [ ]:
indlist = ['pop', 'gdp']
conversions = [(1000000, 2), (1000000000, 2)]
metalist = ['country', 'land']
yearlist = [2000, 2017]
codelist = ['CHN', 'IND', 'USA']

inddatadir = os.path.join(datadir, "worldbank")
df = build_indicators(indlist, conversions, metalist, yearlist, codelist, 
                      aggregates, countrydf, inddatadir)
mixed_table = df.reset_index()
mixed_table.to_csv(os.path.join(datadir, "mixed_table.csv"), index=False)